In [1]:
import random
import time

import models
import numpy as np
import pandas as pd
import torch
import pickle

In [2]:
with open('./dw_i-i_test_result.pkl', 'rb') as f:
    dw_i2i = pickle.load(f)

In [3]:
with open('./dw_test_result.pkl', 'rb') as f:
    dw = pickle.load(f)

In [4]:
with open('./rand_test_result.pkl', 'rb') as f:
    rand = pickle.load(f)

In [5]:
with open('./txt_test_result.pkl', 'rb') as f:
    txt = pickle.load(f)

In [6]:
results = {
    'dw': dw,
    'dw_i2i': dw_i2i,
    'rand': rand,
    'txt': txt
}

In [7]:
dfs = []
for name, gru in results.items():
    scores = torch.from_numpy(gru[2])
    scores = torch.softmax(scores, dim=1)
    gru_result = pd.DataFrame({
        'sess_id': gru[0].repeat(100),
        'sku_id': gru[1].flatten(),
        f'{name}_scores': scores.numpy().flatten()
    })
    dfs
    dfs.append(gru_result)

In [8]:
mix=None
for method in dfs[1:]:
    if mix is None:
        mix = dfs[0].merge(method, on=['sess_id', 'sku_id'], how='outer')
    else:
        mix = mix.merge(method, on=['sess_id', 'sku_id'], how='outer')
mix.fillna(0, inplace=True)

In [9]:
mix

,sess_id,sku_id,dw_scores,dw_i2i_scores,rand_scores,txt_scores
0,5076755,29056,0.311003,0.280207,0.261168,0.201701
1,5076755,47338,0.160307,0.176452,0.214327,0.171656
2,5076755,33952,0.096435,0.138846,0.107644,0.128306
3,5076755,46257,0.093429,0.101615,0.078498,0.099906
4,5076755,15616,0.052467,0.037509,0.036763,0.058752
...,...,...,...,...,...,...
40047211,5408428,14616,0.000000,0.000000,0.000000,0.001654
40047212,5408428,9794,0.000000,0.000000,0.000000,0.001631
40047213,5408428,11540,0.000000,0.000000,0.000000,0.001587
40047214,5408428,16896,0.000000,0.000000,0.000000,0.001583


In [10]:
mix['scores'] = mix['dw_scores'] * 0.5 + mix['dw_i2i_scores'] * 0.5 + mix['rand_scores'] * 0.3 + mix['txt_scores'] * 0.3
mix.sort_values('scores', ascending=False, inplace=True)

In [11]:
mix

,sess_id,sku_id,dw_scores,dw_i2i_scores,rand_scores,txt_scores,scores
13615100,5277530,2670,0.984904,0.983736,0.949967,0.922346,1.546014e+00
8275400,5198847,2670,0.984904,0.983736,0.949967,0.922346,1.546014e+00
7896700,5193269,2670,0.964781,0.984404,0.915739,0.917005,1.524416e+00
10856700,5237014,38553,0.946147,0.906409,0.912797,0.870733,1.461337e+00
4024100,5136028,38553,0.946147,0.906409,0.912797,0.870733,1.461337e+00
...,...,...,...,...,...,...,...
29999217,5130063,41840,0.000000,0.000000,0.000002,0.000000,6.372613e-07
29999218,5130063,23030,0.000000,0.000000,0.000002,0.000000,6.350675e-07
29999219,5130063,20500,0.000000,0.000000,0.000002,0.000000,6.279179e-07
29999220,5130063,36263,0.000000,0.000000,0.000002,0.000000,6.215121e-07


In [12]:
result = mix

In [13]:
sub_result = result.groupby('sess_id', as_index=False).apply(lambda x: x.head(20)).reset_index()
sub_result = sub_result.groupby('sess_id')['sku_id'].agg(list).reset_index()

In [14]:
uids = sub_result['sess_id'].values
iids = sub_result['sku_id'].values

In [15]:
import json
import tqdm

In [16]:
class EasyDict():
    def __init__(self, d):
        for k, v in d.items():
            setattr(self, k, v)


In [17]:
with open('./dataset/raw/rec_test_phase_2.json') as json_file:
        # read the test cases from the provided file
        test_queries = json.load(json_file)

with open('./dataset/new/map_info.pkl', 'rb') as f:
    # read the test cases from the provided file
    info = EasyDict(pickle.load(f))

with open('./results/deepwalk_i_s_u.pkl', 'rb') as f:
    # read the test cases from the provided file
    dw_uids, dw_iids = pickle.load(f)

uids = [info.idx2sess[uid] for uid in uids]
iids = [[info.idx2item[iid] for iid in ilst] for ilst in iids]
preds = dict(zip(uids, iids))

dw_uids = [info.idx2sess[uid] for uid in dw_uids]
dw_iids = [[info.idx2item[iid] for iid in ilst] for ilst in dw_iids]
dw_preds = dict(zip(dw_uids, dw_iids))

all_items = list(info.item2idx.keys())
my_predictions = []
missing = 0
for t in tqdm.tqdm(test_queries, total=len(test_queries)):
    # this is our prediction, which defaults to a random SKU
    next_sku = np.random.choice(len(all_items), 20)
    next_sku = [info.idx2item[iid] for iid in next_sku]
    # copy the test case
    _pred = dict(t)

    session_id_hash = t['query'][0]['session_id_hash']
    if session_id_hash in preds:
        next_sku = preds[session_id_hash]
    elif session_id_hash in dw_preds:
        next_sku = dw_preds[session_id_hash]
    else:
        missing += 1

    # assert isinstance(next_sku, str)

    # append the label - which needs to be a list
    _pred["label"] = next_sku
    # append prediction to the final list
    my_predictions.append(_pred)

print('缺失比例:{}'.format(missing / len(test_queries)))

100%|██████████| 332247/332247 [00:12<00:00, 26089.77it/s]

缺失比例:0.001724620538334432


In [18]:
# name the prediction file according to the README specs
local_prediction_file = '{}_{}.json'.format(
    'tsotfsk_bupt.edu.cn', round(time.time() * 1000))

# dump to file
with open(local_prediction_file, 'w') as outfile:
    json.dump(my_predictions, outfile, indent=2)

In [19]:
local_prediction_file

'tsotfsk_bupt.edu.cn_1623973634959.json'

In [ ]:
#     # finally, upload the test file using the provided script
#     upload_submission(local_file=local_prediction_file, task='rec')
#     # bye bye
#     print("\nAll done at {}: see you, space cowboy!".format(
#         datetime.datetime.utcnow()))